![alt text](1.png "Title")

In [2]:
pip install pandas

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### <font color=blue>1. Maximum Likelihood</font> ###

__Report Mean and Covariance values for each class__

In [11]:
import pandas as pd

star_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/star_dataset.csv')

star_dataset = star_dataset[(star_dataset['Spectral Class'] == 'B') | (star_dataset['Spectral Class'] == 'O')]

star_dataset_2features = star_dataset[['Temperature (K)','Absolute magnitude(Mv)']]

X_features = star_dataset_2features.to_numpy()

In [12]:
X = star_dataset
X0 = X[X['Spectral Class'] == 'B']
X0 = X0[['Temperature (K)','Absolute magnitude(Mv)']]

X1 = X[X['Spectral Class'] == 'O']
X1 = X1[['Temperature (K)','Absolute magnitude(Mv)']]

X0 = X0.to_numpy()
X1 = X1.to_numpy()

In [37]:
from sklearn.covariance import empirical_covariance
import numpy as np

print('X0 Covariance:')
print(empirical_covariance(X0))
print('\n')
print('X0_\u03BC = [', '{:.2f}'.format(np.mean(X0[:,0])),'{:.2f}'.format(np.mean(X0[:,1])),']')
print('\n')
print('X1 Covariance:')
print(empirical_covariance(X1))
print()
print('X1_\u03BC = [', '{:.2f}'.format(np.mean(X1[:,0])),'{:.2f}'.format(np.mean(X1[:,1])),']')

X0 Covariance:
[[ 3.75467284e+07 -2.67843319e+04]
 [-2.67843319e+04  7.51544584e+01]]


X0_μ = [ 19574.48 3.72 ]


X1 Covariance:
[[ 1.09342188e+08 -6.18115637e+02]
 [-6.18115637e+02  2.23654314e+00]]

X1_μ = [ 22294.08 -6.60 ]


### <font color=blue>2. Bayesian Density Estimation</font> ###
__1. Report Mean and Covariance values for each class__<br>

In [38]:
from sklearn.mixture import BayesianGaussianMixture
BDE = BayesianGaussianMixture(n_components=2,covariance_type='full').fit(X_features)
print('\u03BC = ', BDE.means_, sep="\n")
print('\u03A3 = ', BDE.covariances_, sep="\n")

μ = 
[[ 1.69991167e+04  1.12995092e+01]
 [ 2.23633696e+04 -5.98819174e+00]]
Σ = 
[[[ 1.72475007e+07 -4.40571850e+03]
  [-4.40571850e+03  9.39795304e+00]]

 [[ 8.50458239e+07 -1.97805139e+03]
  [-1.97805139e+03  5.75522208e+00]]]


### <font color=blue>3. Expectation Maximization</font> ###
__1. Report Mean and Covariance values for each class__<br>
__2. Plot the Clusters__

In [39]:
from sklearn.mixture import GaussianMixture

GMM = GaussianMixture(n_components=2, covariance_type='full').fit(X_features)
print('Converged:',GMM.converged_)
means = GMM.means_ 
covariances = GMM.covariances_

print('\u03BC = ', means, sep="\n")
print('\u03A3 = ', covariances, sep="\n")

Converged: True
μ = 
[[ 1.68391667e+04  1.18154167e+01]
 [ 2.23878871e+04 -6.06737097e+00]]
Σ = 
[[[ 1.49760780e+07 -1.80027465e+03]
  [-1.80027465e+03  6.74842493e-01]]

 [[ 8.65636517e+07 -1.56714093e+03]
  [-1.56714093e+03  4.44045265e+00]]]


### <font color=blue>4. Gaussian HMM</font><br> ###

__1. Write the Transition Matrix and Emission Matrix__<br><br>
__2. What is the probability of <font color=green>Sunny, Rainy, Rainy, Rainy, Snowy, Snowy</font>? (No need to code!)__<br><br>
__3. Suppose that a man wore <font color=green>Coat, Coat, Umbrella, Umbrella, T-shirt, Umbrella, Coat</font>. What is the most likely weather condition in these 7 consecutive days?__

In [42]:
pip install hmmlearn

     |████████████████████████████████| 368kB 4.3MB/s 


In [43]:
import numpy as np
from hmmlearn import hmm
    
states = ["Sunny", "Rainy", "Snowy"]
n_states = len(states)

observations = ["T-shirt", "Coat", "Umberlla"]
n_observations = len(observations)

In [44]:
model = hmm.MultinomialHMM(n_components = n_states, init_params= "", params= "")

model.startprob_ = np.array([0.7, 0.25, 0.05])    

model.transmat_ = np.array([
                            [0.8, 0.15, 0.05], 
                            [0.38, 0.6, 0.02],
                            [0.75, 0.05, 0.2]
                            ])  
                        
model.emissionprob_ = np.array([
                                [0.6, 0.3, 0.1],
                                [0.05, 0.3, 0.65],
                                [0, 0.5, 0.5]
                              ])

In [45]:
# predict a sequence of hidden states based on visible states
sto = np.array([[1, 1, 2, 2, 0, 2, 1]]).T

model = model.fit(sto)
logprob, hs = model.decode(sto, algorithm="viterbi")
print("Observation:", sto.T)
print("States:", hs)

print("Color of the shirt is:", ", ".join(map(lambda x: observations[int(x)], sto)))
print("States are:", ", ".join(map(lambda x: states[int(x)], hs)))

Observation: [[1 1 2 2 0 2 1]]
States: [1 1 1 1 0 0 0]
Color of the shirt is: Coat, Coat, Umberlla, Umberlla, T-shirt, Umberlla, Coat
States are: Rainy, Rainy, Rainy, Rainy, Sunny, Sunny, Sunny


In [46]:
Gmodel = hmm.GaussianHMM(n_components = 3, covariance_type ="full", init_params= "")

Gmodel.startprob_ = np.array([0.7, 0.25, 0.05])

Gmodel.transmat_ = np.array([
                            [0.8, 0.15, 0.05], 
                            [0.38, 0.6, 0.02],
                            [0.75, 0.05, 0.2]
                            ])
    
Gmodel.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]]) #Assumption

Gmodel.covars_ = np.tile(np.identity(2), (3, 1, 1)) #Assumption

X, Z = Gmodel.sample(10, random_state= 4)   #Generating samples

print("Observation:", X, sep="\n")
print("States:", Z, sep="\n")

Observation:
[[ 5.45036151 10.04500167]
 [-0.99590893  0.69359851]
 [ 6.45162561  9.66118802]
 [ 0.33225003 -1.14747663]
 [-0.48492332 -0.12066567]
 [ 5.61866969  9.91201307]
 [ 2.81959764 -2.79781988]
 [-1.15681626  0.35099715]
 [ 0.74981904 -2.4598148 ]
 [ 2.01700835 -2.94556726]]
States:
[2 0 2 0 0 2 1 0 0 1]
